# Molekülmodell

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In Erweiterung des Jupyter-Notebooks [5-03-Atommodell.ipynb](5-03-Atommodell.ipynb), in dem wir ein eindimensionales Modellpotential für ein Atom betrachtet haben, untersuchen wir nun ein eindimensionales Modellpotential

$$ V(x) = - \frac{\alpha^3}{\sqrt{\alpha^2+(x+\Delta/2)^2}} - \frac{\alpha^3}{\sqrt{\alpha^2+(x-\Delta/2)^2}} $$

für ein zweiatomiges Molekül. Die Berechnung der Eigenenergien und Eigenvektoren erfolgt wie in den Jupyter-Notebooks [5-02-Harmonischer-Oszillator.ipynb](5-02-Harmonischer-Oszillator.ipynb) und [5-03-Atommodell.ipynb](5-03-Atommodell.ipynb). Am Ende dieses Jupyter-Notebooks wird zusätzlich noch untersucht, wie sich die niedrigsten Eigenenergien als Funktion des Abstands $\Delta$ zwischen den Potentialminima verhalten.

## Importanweisungen

In [ ]:
import numpy as np
import numpy.linalg as LA
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Energetisch niedrigste Eigenzustände

### Berechnung des Potentials

Durch die Verwendung der Wurzelfunktion aus dem NumPy-Paket kann die Funktion `potential` neben skalaren Werten für das Argument `x` auch NumPy-Arrays verarbeiten.

In [2]:
def potential(x, alpha, delta):
    return -alpha**3*(1/np.sqrt(alpha**2+(x+delta/2)**2)
                      + 1/np.sqrt(alpha**2+(x-delta/2)**2))

### Berechnung des Hamilton-Operators

Diese Funktion ist identisch mit der Funktion `hamilton_operator` aus dem Jupyter-Notebook [5-02-Harmonischer-Oszillator.ipynb](5-02-Harmonischer-Oszillator.ipynb).

In [ ]:
def hamilton_operator(n_max, x_max, alpha, delta):
    ndim = 2*n_max+1
    x, dx = np.linspace(-x_max, x_max, ndim, retstep=True)
    h = (np.eye(ndim)
         - 0.5*(np.eye(ndim, k=1) + np.eye(ndim, k=-1)))
    h = h/dx**2 + np.diag(potential(x, alpha, delta))
    return h

### Berechnung der Eigenwerte und Eigenvektoren

Diese Funktion ist identisch mit der Funktion `eigenproblem` aus dem Jupyter-Notebook [5-02-Harmonischer-Oszillator.ipynb](5-02-Harmonischer-Oszillator.ipynb).

In [ ]:
def eigenproblem(n_max, x_max, alpha, delta):
    h = hamilton_operator(n_max, x_max, alpha, delta)
    evals, evecs = LA.eigh(h)
    return evals, evecs

### Implementierung der Bedienelemente und graphische Darstellung der sechs niedrigsten Eigenzustände

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `n_max`: Anzahl der Gitterpunkte auf einer Seite des Ursprungs. Insgesamt werden `2*n_max+1` Gitterpunkte verwendet.
- `x_max`: Ausdehnung des Gitters von `-x_max` bis `x_max`
- `alpha`: Potentialparameter
- `delta`: Abstand der Potentialminima

Da das Verhalten der energetisch niedrigsten Eigenzustände als Funktion der Barrierenhöhe am interessantesten ist, wird die Darstellung auf sechs Eigenzustände beschränkt. Bei Bedarf kann die Anzahl durch Anpassung der Variable `n_states` zu Beginn der Funktion `plot_eigenstates` angepasst werden.

In [ ]:
widget_dict = {"n_max":
               widgets.IntSlider(
                   value=500, min=10, max=1000, step=10,
                   description=r"$n_\text{max}$"),
               "x_max":
               widgets.FloatSlider(
                   value=80, min=10, max=100, step=1,
                   description=r"$x_\text{max}$"),
               "alpha":
               widgets.FloatSlider(
                   value=2, min=0.1, max=10, step=0.1,
                   description=r"$\alpha$"),
               "delta":
               widgets.FloatSlider(
                   value=8, min=1, max=20, step=0.5,
                   description=r"$\Delta$")
               }


interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_eigenstates(n_max, x_max, alpha, delta):
    n_states = 6
    evals, evecs = eigenproblem(n_max, x_max, alpha, delta)
    x_values = np.linspace(-x_max, x_max, 2*n_max+1)

    fig, ax = plt.subplots()
    for n in range(n_states):
        ax.plot(x_values, evecs[:, n]/2+evals[n],
                label=f"E={evals[n]:.5f}")
    ax.plot(x_values, potential(x_values, alpha, delta),
            color="black")
    ax.set_xlim((-2*delta, 2*delta))
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$\psi(x)$")
    ax.legend(bbox_to_anchor=(0.5, 1.01),
              loc="lower center", ncol=2)

## Abstandsabhängigkeit der Eigenenergien

### Berechnung der Eigenenergien als Funktion des Abstands zwischen den Potentialminima

Zur Berechnung der Eigenenergien wird statt `LA.eigh` die Funktion `LA.eigvalsh` aus der NumPy-Bibliothek verwendet. Dadurch wird die hier unnötige Berechnung der Eigenvektoren vermieden. Diese Funktion ist für hermitesche Matrizen gedacht und garantiert reelle Eigenwerte, die in aufsteigender Reihenfolge geordnet sind. Das zweidimensionale NumPy-Array `energies` enthält zu jedem Wert des Abstands $\Delta$ eine Zeile mit den berechneten Eigenwerten.

In [ ]:
def lowest_energies_over_delta(n_max, x_max, alpha,
                               delta_values, n_states):
    energies = np.zeros((len(delta_values), n_states))
    for n_delta, delta in enumerate(delta_values):
        h = hamilton_operator(n_max, x_max, alpha, delta)
        evals = LA.eigvalsh(h)
        energies[n_delta, :] = evals[0:n_states]
    return energies

### Implementierung der Bedienelemente und graphische Darstellung der Abstandsabhängigkeit der niedrigsten Eigenenergien

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `n_max`: Anzahl der Gitterpunkte auf einer Seite des Ursprungs. Insgesamt werden `2*n_max+1` Gitterpunkte verwendet.
- `x_max`: Ausdehnung des Gitters von `-x_max` bis `x_max`
- `alpha`: Potentialparameter
- `delta_max`: maximaler Abstand der Potentialminima
- `n_delta_max`: Anzahl der zu betrachtenden Abstandswerte $\Delta$

Wie zuvor bei den Eigenzuständen zu festem $\Delta$ werden hier die sechs niedrigsten Eigenenergien betrachtet. Der Wert kann bei Bedarf zu Beginn der Funktion `plot_energy_over_delta` mit Hilfe der Variable `n_states` angepasst werden.

In [ ]:
widget_dict = {"n_max":
               widgets.IntSlider(
                   value=500, min=10, max=1000, step=10,
                   description=r"$n_\text{max}$"),
               "x_max":
               widgets.FloatSlider(
                   value=80, min=10, max=100, step=1,
                   description=r"$x_\text{max}$"),
               "alpha":
               widgets.FloatSlider(
                   value=2, min=0.1, max=10, step=0.1,
                   description=r"$\alpha$"),
               "delta_max":
               widgets.FloatSlider(
                   value=20, min=1, max=20, step=0.1,
                   description=r"$\Delta_\text{max}$"),
               "n_delta_max":
               widgets.IntSlider(
                   value=100, min=10, max=500, step=10,
                   description=r"$n_\Delta$")
               }

@interact_start(**widget_dict)
def plot_energy_over_delta(n_max, x_max, alpha, delta_max,
                           n_delta_max):
    n_states = 6
    delta_values = np.linspace(0, delta_max, n_delta_max)
    lowest_energies = lowest_energies_over_delta(
        n_max, x_max, alpha, delta_values, n_states)

    fig, ax = plt.subplots()
    for n in range(n_states):
        ax.plot(delta_values, lowest_energies[:, n])
    ax.set_xlabel(r"$\Delta$")
    ax.set_ylabel("$E_0$")
    ax.set_xlim((0, delta_max))